MODEL HAZIRLAMA VE EĞİTİM

In [737]:
import numpy as np
from numpy import genfromtxt

In [738]:
x_eğitim = genfromtxt('eğitim_data.csv', delimiter=',')
y_eğitim = genfromtxt('eğitim_labels.csv', delimiter=',')
x_test = genfromtxt('test_data.csv', delimiter=',')
y_test = genfromtxt('test_labels.csv', delimiter=',')

In [739]:
len(x_eğitim)

7895

In [740]:
len(x_test)

837

In [741]:
len(y_eğitim)

7895

In [742]:
len(y_test)

837

In [743]:
x_doğrulama= x_eğitim[7000:]
y_doğrulama = y_eğitim[7000:]

In [744]:
x_eğitim= x_eğitim[:7000]
y_eğitim= y_eğitim[:7000]

In [745]:
len(x_doğrulama)

895

In [746]:
len(x_eğitim)

7000

In [747]:
from keras.utils.np_utils import to_categorical
y_eğitim = to_categorical(y_eğitim, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)
y_doğrulama = to_categorical(y_doğrulama, num_classes=10)
y_eğitim.shape,y_test.shape,y_doğrulama.shape

((7000, 10), (837, 10), (895, 10))

In [748]:
x_eğitim=np.reshape(x_eğitim,(x_eğitim.shape[0], 36,1))
x_test=np.reshape(x_test,(x_test.shape[0], 36,1))
x_doğrulama=np.reshape(x_doğrulama,(x_doğrulama.shape[0], 36,1))
x_eğitim.shape,x_test.shape,x_doğrulama.shape

((7000, 36, 1), (837, 36, 1), (895, 36, 1))

In [749]:
x_eğitim=np.reshape(x_eğitim,(x_eğitim.shape[0], 36,1,1))
x_test=np.reshape(x_test,(x_test.shape[0], 36,1,1))
x_doğrulama=np.reshape(x_doğrulama,(x_doğrulama.shape[0], 36,1,1))

In [750]:
x_eğitim.shape

(7000, 36, 1, 1)

In [751]:
x_test.shape

(837, 36, 1, 1)

In [752]:
import tensorflow as tf


In [753]:
model = tf.keras.Sequential()

In [754]:
model.add(tf.keras.layers.Conv2D(32,
                                kernel_size=(3,3),
                                strides=(1,1),  
                                padding='same', 
                                activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(padding="same"))


model.add(tf.keras.layers.Conv2D(64,                                
                                kernel_size=(3,3),
                                strides=(1,1),  
                                padding='same', 
                                activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(padding="same"))
model.add(Dropout(0.3))

model.add(tf.keras.layers.Conv2D(128,
                                kernel_size=(3,3),
                                strides=(1,1),  
                                padding='same', 
                                activation='relu')) 
model.add(tf.keras.layers.MaxPooling2D(padding="same"))
model.add(Dropout(0.3))


In [755]:
#"Flatten" yöntemi tüm piksel değerlerini alıp tek tek 1 boyutlu diziye koyuyoruz. 

model.add(tf.keras.layers.Flatten())

In [756]:
#iki yoğun katman ekliyoruz. 
# Her biri 64 düğüm(nodes)

model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))

In [757]:
#Çıktı katmanınını oluşturalım. 
#10 sınıfımız olduğu için 10 düğüm ekliyoruz ve Softmax aktivasyon fonksiyonunu kullanıyoruz. 
#Çok sınıflı sınıflandırma problemleri için çıktı katmanında Softmax'ı t'de kullanırız

model.add(tf.keras.layers.Dense(10, activation='softmax'))

In [758]:
#Çok sınıflı bir sınıflandırma problemini çözmeye çalıştığımız için "Sparse Categorical Cross Entropy" kayıp fonksiyonunu kullanıyoruz. 

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [759]:
results= model.fit(x_eğitim, y_eğitim,
                 batch_size= 72,
                 epochs = 10,
                 validation_data=(x_doğrulama, y_doğrulama))

Epoch 1/10


InvalidArgumentError: ignored

In [ ]:
#loss ve accuracy grafiklerin
import matplotlib.pyplot as plt

plt.plot(results.history['loss'],label='loss')

plt.plot(results.history['val_loss'],label='val_loss')

plt.xlabel('metrics', color='red')
plt.ylabel('loss', color='orange')

plt.legend()

plt.show()

In [ ]:
#Eğitim ve test, kayıp ve puanlar
eğitim_loss_score=model.evaluate(x_eğitim,y_eğitim)
test_loss_score=model.evaluate(x_test,y_test)
print(eğitim_loss_score)
print(test_loss_score)